# Interactive Strategy Analysis

This notebook demonstrates the interactive analysis workflow for ADMF-PC backtest results.

In [ ]:
# Import the interactive analysis tools
import sys
sys.path.append('../..')  # Adjust based on notebook location

from src.analytics.interactive import AnalysisWorkspace, QueryLibrary, PatternLibrary
from src.analytics.notebook_cells import queries, performance, correlation, visualization, regime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Initialize Workspace and List Available Runs

In [ ]:
# Create workspace
workspace = AnalysisWorkspace()

# List recent runs
runs = workspace.list_runs()
runs_df = pd.DataFrame(runs)
runs_df[['run_id', 'config_name', 'total_strategies', 'best_sharpe', 'timeframe']].head(10)

## 2. Load a Specific Run

In [ ]:
# Load the most recent run (or specify a specific one)
run = workspace.load_run(f"results/{runs_df.iloc[0]['run_id']}")

# Show summary
print("Run Summary:")
for key, value in run.summary.items():
    print(f"  {key}: {value}")

## 3. Explore Strategies

In [ ]:
# Get top strategies
top_strategies = workspace.top_strategies(run, n=20)
top_strategies[['strategy_type', 'sharpe_ratio', 'total_return', 'max_drawdown']].head(10)

In [ ]:
# Strategy type distribution
strategy_dist = run.strategies['strategy_type'].value_counts()
plt.figure(figsize=(10, 6))
strategy_dist.plot(kind='bar')
plt.title('Strategy Type Distribution')
plt.xlabel('Strategy Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Custom Queries with DuckDB

In [ ]:
# Example: Find strategies with specific characteristics
custom_query = """
SELECT 
    strategy_type,
    COUNT(*) as count,
    AVG(sharpe_ratio) as avg_sharpe,
    MAX(sharpe_ratio) as max_sharpe,
    AVG(total_return) as avg_return
FROM strategies
WHERE sharpe_ratio > 1.0
GROUP BY strategy_type
ORDER BY avg_sharpe DESC
"""

high_sharpe_strategies = run.query(custom_query)
high_sharpe_strategies

In [ ]:
# Signal frequency analysis
signal_freq = QueryLibrary.signal_frequency(run)
print("\nSignal Frequency Analysis:")
print(signal_freq[['strategy_type', 'total_signals', 'signals_per_day', 'sharpe_ratio']].head(10))

## 5. Correlation Analysis for Ensemble Building

In [ ]:
# Calculate correlation matrix for top strategies
corr_matrix = workspace.correlation_matrix(run, top_strategies.head(10))

# Visualize
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0, 
            annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.title('Strategy Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Find optimal ensemble
ensemble = workspace.find_ensemble(run, size=5, correlation_threshold=0.7)

print(f"Optimal Ensemble:")
print(f"  Average Sharpe: {ensemble['avg_sharpe']:.2f}")
print(f"  Max Correlation: {ensemble['max_correlation']:.2f}")
print("\nEnsemble Strategies:")
print(ensemble['strategies'][['strategy_type', 'sharpe_ratio', 'total_return']])

## 6. Parameter Analysis

In [ ]:
# Analyze parameters for a specific strategy type
strategy_type = top_strategies.iloc[0]['strategy_type']
param_analysis = workspace.analyze_parameters(run, strategy_type)

print(f"\nParameter Analysis for {strategy_type}:")
print(param_analysis)

## 7. Intraday Patterns

In [ ]:
# Analyze intraday patterns
intraday = QueryLibrary.intraday_patterns(run)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(intraday['hour'], intraday['signal_count'])
plt.xlabel('Hour of Day')
plt.ylabel('Signal Count')
plt.title('Trading Activity by Hour')

plt.subplot(1, 2, 2)
plt.plot(intraday['hour'], intraday['avg_direction'], marker='o')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Hour of Day')
plt.ylabel('Average Direction')
plt.title('Signal Direction Bias by Hour')

plt.tight_layout()
plt.show()

## 8. Save Discovered Patterns

In [ ]:
# Save interesting patterns for future use
pattern_lib = PatternLibrary()

# Example: Save a high-performing parameter combination
if len(top_strategies) > 0:
    best_strategy = top_strategies.iloc[0]
    pattern = {
        'strategy_type': best_strategy['strategy_type'],
        'parameters': {col: best_strategy[col] for col in best_strategy.index if col.startswith('param_')},
        'performance': {
            'sharpe_ratio': best_strategy['sharpe_ratio'],
            'total_return': best_strategy['total_return']
        }
    }
    pattern_lib.save_pattern(f"best_{strategy_type}_{run.summary['timeframe']}", pattern)

# List saved patterns
print("\nSaved Patterns:")
for p in pattern_lib.list_patterns():
    print(f"  {p['name']}: discovered {p['discovered']}, used {p['usage_count']} times")

## 9. Custom Analysis Cell

Use the reusable cells from the notebook_cells module:

In [ ]:
# Import and execute a performance analysis cell
exec(performance.sharpe_calculation_cell())

In [ ]:
# Import and execute a regime analysis cell
# First, we need to load market data
market_data = pd.DataFrame()  # You would load actual market data here

# Then run the regime analysis
if not market_data.empty:
    exec(regime.volatility_regime_cell())

## 10. Export Results

In [ ]:
# Export ensemble strategies for production use
ensemble_config = {
    'run_id': run.summary['run_id'],
    'selected_strategies': ensemble['strategies'][['strategy_hash', 'strategy_type']].to_dict('records'),
    'performance': {
        'avg_sharpe': ensemble['avg_sharpe'],
        'max_correlation': ensemble['max_correlation']
    }
}

import json
with open('selected_ensemble.json', 'w') as f:
    json.dump(ensemble_config, f, indent=2)

print("Ensemble configuration saved to selected_ensemble.json")